In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

from nltk.corpus import stopwords
from nltk. tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk

from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/BERTopic/jobstreet_all_job_dataset.csv')

dataset

,job_id,job_title,company,descriptions,location,category,subcategory,role,type,salary,listingDate
0,74630583.0,Procurement Executive (Contract),Coca-Cola Bottlers (Malaysia) Sdn Bhd,Position Purpose\nManage aspects of procuremen...,Negeri Sembilan,"Manufacturing, Transport & Logistics","Purchasing, Procurement & Inventory",procurement-executive,Contract/Temp,NaN,2024-03-21T05:58:35Z
1,74660602.0,Account Executive/ Assistant,Acoustic & Lighting System Sdn Bhd,We are looking for a Account Executive/ Assist...,Petaling,Accounting,Bookkeeping & Small Practice Accounting,executive-assistant,Full time,"RM 2,800 – RM 3,200 per month",2024-03-22T06:52:57Z
2,74655679.0,"Data Analyst - Asset Management, SPX Express",Shopee Mobile Malaysia Sdn Bhd,Performs detailed data analysis on existing sp...,Klang District,"Manufacturing, Transport & Logistics",Analysis & Reporting,asset-management-analyst,Full time,NaN,2024-03-22T04:22:43Z
3,74657624.0,Service Engineer,Sun Medical Systems Sdn Bhd,"You are important for troubleshooting, install...",Petaling,Engineering,Electrical/Electronic Engineering,services-engineer,Full time,"RM 3,000 – RM 3,500 per month",2024-03-22T05:32:09Z
4,74679363.0,Purchasing Executive,Magnet Security & Automation Sdn. Bhd.,"MAG is a trailblazer in the industry, boasting...",Hulu Langat,"Manufacturing, Transport & Logistics","Purchasing, Procurement & Inventory",purchasing-executive,Full time,"RM 2,800 – RM 3,500 per month",2024-03-23T03:56:39Z
...,...,...,...,...,...,...,...,...,...,...,...
59301,76653277.0,"Marketing Specialist, APAC",Glory Global Solutions (M),MAIN RESPONSIBILITIES\nSupport the Asia Pacifi...,Kuala Lumpur,Marketing & Communications,Marketing Communications,marketing-specialist,Full time,"RM 4,000 – RM 6,000 per month",2024-06-18T06:45:54Z
59302,76674488.0,Talent Acquisition Partner,PERSOLKELLY Workforce Solutions Malaysia Sdn Bhd,Job Descriptions:\nPartner closely with HRBPs ...,Kuala Lumpur,Human Resources & Recruitment,Recruitment - Internal,talent-acquisition-partner,Contract/Temp,"RM 4,000 – RM 5,000 per month",2024-06-19T03:37:54Z
59303,76658887.0,Motor Insurance Executive,MRMC Consultants Sdn Bhd,Objective:\nHighly motivated and customer-focu...,Kuala Lumpur,Insurance & Superannuation,Brokerage,Motor-Insurance-Executive,Full time,"RM 2,500 – RM 3,500 per month",2024-06-18T10:15:07Z
59304,76653289.0,Banca Specialist (Klang Valley),AmBank Group,Job Descriptions:\nDelivers sales target for b...,Kuala Lumpur,Banking & Financial Services,Banking - Retail/Branch,specialist,Full time,NaN,2024-06-18T06:46:18Z


In [4]:
# # Select columns useful for analyses
# analyse_columns = [
#     "job_id",
#     "company_name",
#     "title",
#     "description",
#     "original_listed_time",
#     "expiry",
# ]

# dataset = dataset[analyse_columns]

# Select columns useful for analyses
analyse_columns = [
    "job_title",
    "descriptions",
]

dataset = dataset[analyse_columns]

In [5]:
# Data cleaning, fill in missing values
dataset['job_title'].fillna("", inplace=True)
dataset['descriptions'].fillna("", inplace=True)
dataset['descriptions'].fillna("No Description Listed", inplace=True)

# # Convert timestamps to datetime
# dataset['original_listed_time'] = pd.to_datetime(dataset['original_listed_time'], unit='ms')
# dataset['expiry'] = pd.to_datetime(dataset['expiry'], unit='ms')

dataset

<ipython-input-5-b7e9dc6ea445>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['job_title'].fillna("", inplace=True)
<ipython-input-5-b7e9dc6ea445>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['job_title'].fillna("", inplace=True)
<ipython-input-5-b7e9dc6ea445>:3: FutureWarning: A value is trying to be set on a copy of a D

,job_title,descriptions
0,Procurement Executive (Contract),Position Purpose\nManage aspects of procuremen...
1,Account Executive/ Assistant,We are looking for a Account Executive/ Assist...
2,"Data Analyst - Asset Management, SPX Express",Performs detailed data analysis on existing sp...
3,Service Engineer,"You are important for troubleshooting, install..."
4,Purchasing Executive,"MAG is a trailblazer in the industry, boasting..."
...,...,...
59301,"Marketing Specialist, APAC",MAIN RESPONSIBILITIES\nSupport the Asia Pacifi...
59302,Talent Acquisition Partner,Job Descriptions:\nPartner closely with HRBPs ...
59303,Motor Insurance Executive,Objective:\nHighly motivated and customer-focu...
59304,Banca Specialist (Klang Valley),Job Descriptions:\nDelivers sales target for b...


In [6]:
# Preprocessing function for text
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Tokenize and clean text
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]

    return ' '.join(tokens)

In [7]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
# Initialize tqdm for pandas operations
tqdm.pandas()

# Apply preprocessing to the description column with a progress bar
print("Preprocessing descriptions...")
dataset['cleaned_description'] = dataset['descriptions'].progress_apply(preprocess_text)

# Apply preprocessing to the title column with a progress bar
print("Preprocessing titles...")
dataset['cleaned_title'] = dataset['job_title'].progress_apply(preprocess_text)

print("Preprocessing complete.")

Preprocessing descriptions...


100%|██████████| 59306/59306 [02:39<00:00, 370.97it/s]
<ipython-input-9-116500fb32c7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['cleaned_description'] = dataset['descriptions'].progress_apply(preprocess_text)


Preprocessing titles...


100%|██████████| 59306/59306 [00:10<00:00, 5486.55it/s]

Preprocessing complete.



<ipython-input-9-116500fb32c7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['cleaned_title'] = dataset['job_title'].progress_apply(preprocess_text)


In [10]:
dataset.to_csv('/content/drive/My Drive/Colab Notebooks/Comparing Transformer models/preprocessed_jobstreet_dataset.csv', index=False)